<a href="https://colab.research.google.com/github/sggyuan/GNN/blob/main/***%E6%95%B0%E6%8D%AE%E9%A2%84%E5%A4%84%E7%90%86%E4%BB%A5%E5%8F%8A%E8%BD%AC%E5%8C%96%E5%9B%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/My Drive/subset10w.csv')

In [5]:
df["card_id"] = df["User"].astype(str) + "_" + df["Card"].astype(str)
df["Amount"]=df["Amount"].str.replace("$","").astype(float)
df["Hour"] = df["Time"].str [0:2]
df["Minute"] = df["Time"].str [3:5]
df = df.drop(["Time","User","Card"],axis=1)
df["Errors?"]= df["Errors?"].fillna("No error")
df = df.drop(columns=["Merchant State","Zip","Apartment"],axis=1)
df["Is Fraud?"] = df["Is Fraud?"].apply(lambda x: 1 if x == 'Yes' else 0)
df['Per Capita Income - Zipcode'] = df['Per Capita Income - Zipcode'].str.replace('$', '').astype(float)
df['Yearly Income - Person'] = df['Yearly Income - Person'].str.replace('$', '').astype(float)
df['Total Debt'] = df['Total Debt'].str.replace('$', '').astype(float)
categorical_columns = ['Merchant City', 'Errors?' , 'Person', 'Address', 'City', 'State']

le = LabelEncoder()
for col in categorical_columns:
    df[col] = le.fit_transform(df[col].astype(str))

gender_encoded = pd.get_dummies(df['Gender'], prefix='Gender')
df = pd.concat([df, gender_encoded], axis=1)
df = df.drop(['Gender'], axis=1)

bool_columns = df.select_dtypes(include=['bool']).columns
for col in bool_columns:
    df[col] = df[col].astype(int)

df["Use Chip"]=LabelEncoder().fit_transform(df["Use Chip"])

In [8]:
import pandas as pd

# List of all categorical columns to be one-hot encoded
categorical_columns = ['Merchant City', 'Errors?', 'Person', 'Address', 'City', 'State', 'Use Chip']

# Apply one-hot encoding to all specified categorical columns
df_encoded = pd.get_dummies(df, columns=categorical_columns, prefix=categorical_columns)

# Convert boolean columns to integers
bool_columns = df.select_dtypes(include=['bool']).columns
for col in bool_columns:
    df_encoded[col] = df_encoded[col].astype(int)

print(df_encoded.head())
print(df_encoded.shape)

   Year  Month  Day  Amount        Merchant Name   MCC  Is Fraud?  \
0  2018      8    8   86.89 -5162038175624867091  5541          0   
1  2010      9   28    1.91  6091778774361517457  5411          0   
2  2009      2   23   13.82  2027553650310142703  5541          0   
3  2013     12   10   15.34 -4317138273541964845  5211          0   
4  2013     11   30   14.82 -5581123930363301609  5311          0   

   Current Age  Retirement Age  Birth Year  ...  State_44  State_45  State_46  \
0           57              66        1962  ...     False     False     False   
1           66              69        1954  ...     False     False     False   
2           49              59        1970  ...     False     False     False   
3           33              67        1986  ...     False     False     False   
4           40              70        1979  ...     False     False     False   

   State_47  State_48  State_49  State_50  Use Chip_0  Use Chip_1 Use Chip_2  
0     False     Fal

In [6]:
# Create an empty graph
G = nx.MultiGraph()

# Add nodes to the graph for each unique card_id, merchant_name
G.add_nodes_from(df["card_id"].unique(), type='card_id')
G.add_nodes_from(df["Merchant Name"].unique(), type='merchant_name')

# Add nodes to the graph for each unique card_id and merchant_name
for _, row in df.iterrows():
    # Add card_id node with its properties
    G.add_node(row['card_id'],
               person=row['Person'],
               current_age=row['Current Age'],
               retirement_age=row['Retirement Age'],
               birth_year=row['Birth Year'],
               birth_month=row['Birth Month'],
               gender_0=row['Gender_Female'],
               gender_1=row['Gender_Male'],
               address=row['Address'],
               city=row['City'],
               state=row['State'],
               zipcode=row['Zipcode'],
               latitude=row['Latitude'],
               longitude=row['Longitude'],
               per_capita_income=row['Per Capita Income - Zipcode'],
               yearly_income=row['Yearly Income - Person'],
               total_debt=row['Total Debt'],
               fico_score=row['FICO Score'],
               num_credit_cards=row['Num Credit Cards'])


     # Add merchant_name node with its properties
    G.add_node(row['Merchant Name'],
               merchant_city=row['Merchant City'],
               mcc=row['MCC'])

# Add edges and properties to the edges
for _, row in df.iterrows():
    G.add_edge(row['card_id'], row['Merchant Name'],
               year=row['Year'],
               month=row['Month'],
               day=row['Day'],
               hour=row['Hour'],
               minute=row['Minute'],
               amount=row['Amount'],
               use_chip=row['Use Chip'],
               errors=row['Errors?'],
               is_fraud=row['Is Fraud?'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 31 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Year                         100000 non-null  int64  
 1   Month                        100000 non-null  int64  
 2   Day                          100000 non-null  int64  
 3   Amount                       100000 non-null  float64
 4   Use Chip                     100000 non-null  int64  
 5   Merchant Name                100000 non-null  int64  
 6   Merchant City                100000 non-null  int64  
 7   MCC                          100000 non-null  int64  
 8   Errors?                      100000 non-null  int64  
 9   Is Fraud?                    100000 non-null  int64  
 10  Person                       100000 non-null  int64  
 11  Current Age                  100000 non-null  int64  
 12  Retirement Age               100000 non-null  int64  
 13  

In [7]:
import torch
from torch_geometric.data import Data
import networkx as nx

def convert_to_pyg(G):
    node_features = []
    node_mapping = {}

    # 定义卡片节点和商家节点的特征列表
    card_features = ['current_age', 'retirement_age', 'birth_year', 'birth_month', 'gender_0', 'gender_1',
                     'per_capita_income', 'yearly_income', 'total_debt', 'fico_score', 'num_credit_cards']
    merchant_features = ['mcc']

    # 获取所有可能的特征
    all_features = card_features + merchant_features + ['is_card']  # 'is_card' 是类型标志位

    for i, (node, data) in enumerate(G.nodes(data=True)):
        node_mapping[node] = i
        features = []

        if 'person' in data:  # 卡片节点
            features = [float(data.get(f, 0)) for f in card_features]
            features += [0] * len(merchant_features)  # 填充商家特征
            features.append(1)  # 标记为卡片节点
        else:  # 商家节点
            features = [0] * len(card_features)  # 填充卡片特征
            features += [float(data.get(f, 0)) for f in merchant_features]
            features.append(0)  # 标记为商家节点

        node_features.append(features)

    # 创建边索引和边属性
    edge_index = []
    edge_attr = []
    for source, target, data in G.edges(data=True):
        edge_index.append([node_mapping[source], node_mapping[target]])
        edge_attr.append([
            float(data.get('year', 0)), float(data.get('month', 0)), float(data.get('day', 0)),
            float(data.get('hour', 0)), float(data.get('minute', 0)),
            float(data.get('amount', 0)), float(data.get('use_chip', 0)), float(data.get('errors', 0))
        ])

    # 创建 PyG 数据对象
    x = torch.FloatTensor(node_features)
    edge_index = torch.LongTensor(edge_index).t().contiguous()
    edge_attr = torch.FloatTensor(edge_attr)
    y = torch.LongTensor([int(data.get('is_fraud', 0)) for _, _, data in G.edges(data=True)])

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

# 转换 NetworkX 图为 PyG 数据
data = convert_to_pyg(G)
print(data)

# 打印特征维度以验证
print(f"Node feature dimensions: {data.x.shape}")

Data(x=[16283, 13], edge_index=[2, 100000], edge_attr=[100000, 8], y=[100000])
Node feature dimensions: torch.Size([16283, 13])


In [8]:
print("PyG Data object:", data)

torch.save(data, '/content/drive/MyDrive/my_graph_data-bal.pt')

PyG Data object: Data(x=[16283, 13], edge_index=[2, 100000], edge_attr=[100000, 8], y=[100000])
